In [2]:
!pip install numexpr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 863.8 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [11]:
%reload_ext autoreload
%autoreload 2

In [12]:
import os
import numpy as np
import pandas as pd
import datasets
import numexpr  # noqa: F401
import math
from transformers import Tool
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [42]:
class CalculatorTool(Tool):
    name = "calculator"
    description = "This is a tool that calculates. It can be used to perform simple arithmetic operations."

    inputs = ["text"]
    outputs = ["float"]

    def __call__(self, expression):
        local_dict = {"pi": math.pi, "e": math.e}
        output = str(
            numexpr.evaluate(
                expression.strip(),
                global_dict={},  # restrict access to globals
                local_dict=local_dict,  # add common mathematical functions
            )
        )
        return output

In [43]:
def split_answer(row):
    splitted = row["answer"].split("####")
    row["true_reasoning"] = splitted[0]
    row["true_answer"] = float(splitted[1].strip())
    return row


math_dataset = (
    datasets.load_dataset("gsm8k", "main")["train"].shuffle(seed=42).select(range(100))
)
math_dataset = pd.DataFrame(math_dataset)
math_dataset = math_dataset.apply(split_answer, axis=1)
math_dataset = math_dataset.drop(columns=["answer"])
math_dataset = datasets.Dataset.from_pandas(math_dataset)

eval_dataset = math_dataset.select(range(30))

In [51]:
def fake_llm(prompt, stop=None):
    if "marker" not in prompt:
        return """
        Thought: I should multiply 2 by 3.6452. marker
        Action:
        {
            "action": "calculator",
            "action_input": {"expression": "2*3.6452"}
        }
        """
    else:  # We're at step 2
        return """
            Thought: I can now answer the initial question
            Action:
            {
                "action": "final_answer",
                "action_input": {"expression": "1.234567"}
            }
            """

In [52]:
from transformers import Agent

agent = Agent(
    llm_engine=fake_llm,
    toolbox=[CalculatorTool()],
)

In [56]:
output = agent.run("What is 2 multiplied by 3.6452?")

==Thought from the agent==

            Thought: I can now answer the initial question
            


In [57]:
output

{'expression': '1.234567'}